In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LassoCV,RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
import math

/Users/liutianhan/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Read In Data

In this section, our goal is to generate processed training and test sets. First, we read in our data from csv file. We split the data into training and test, impute missing values through the mean and finally we transform response variables using log.

In [2]:
# Read-in the full data set
data = pd.read_csv('Final_Dataframe.csv')
data = data.drop(["Unnamed: 0"], axis=1)

In [3]:
data.head()

,Unnamed: 0,acousticness_mean,acousticness_std,dance_mean,dance_std,energy_mean,energy_std,instrumentalness_mean,instrumentalness_std,key_mean,key_std,liveness_mean,liveness_std,loudness_mean,loudness_std,mode_mean,mode_std,speech_mean,speech_std,tempo_mean,tempo_std,time_mean,time_std,valence_mean,valence_std,Followers,followers_mean,followers_std,popularity_mean,popularity_std,top_0_10,top_10_20,top_20_30,top_30_40,top_40_50,'acid house','album rock','alternative country','alternative dance','alternative metal','alternative pop','alternative rock','alternative roots rock','ambient','anthem emo','anthem worship','anti-folk','art rock','athens indie','austindie','australian alternative rock','australian dance','australian hip hop','australian pop','avant-garde','azonto','azontobeats','bass music','bay area indie','bebop','big band','big beat','big room','bluegrass','blues','blues-rock','boogie-woogie','bossa nova','boston rock','bow pop','breakbeat','brill building pop','british alternative rock','british blues','british folk','british indie rock','british invasion','britpop','brooklyn indie','brostep','bubblegum pop','c86','cabaret','canadian indie','canadian metal','canadian pop','candy pop','canterbury scene','catstep','ccm','cello','celtic rock','chamber pop','chamber psych','chaotic hardcore','chicago blues','chicago house','chicago indie','chicago soul','chillhop',...,'nu age','nu metal','permanent wave','pixie','polka','pop emo','pop rap','pop rock','pop','post-screamo','post-teen pop','progressive deathcore','progressive post-hardcore','psychedelic doom','punk','quebecois','rap','redneck','reggae fusion','reggae','reggaeton','relaxative','scorecore','sheffield indie','shimmer pop','shiver pop','singer-songwriter','skate punk','slow core','slow game','spanish pop','speed garage','talent show','traditional country','trap latino','trap music','trip hop','tropical house','uk drill','uk hip hop','underground hip hop','vancouver indie','vapor soul','vegas indie','violin','viral pop','west coast trap','wrestling','no_genre',Lil Wayne,Van Morrison,Galantis,Wiz Khalifa,Rihanna,Post Malone,Axwell /\ Ingrosso,Young Thug,JAY Z,A$AP Rocky,Yo Gotti,Chance The Rapper,Led Zeppelin,Otis Redding,21 Savage,Deorro,Elton John,SZA,Ty Dolla $ign,Ryan Adams,Birdy,Miguel,Niall Horan,Ellie Goulding,Commodores,Radiohead,SYML,First Aid Kit,Lord Huron,Str_Best,Str_Workout,Str_Party,Str_Chill,Str_Acoustic,Str_2000s,Str_1990s,Str_1980s,Str_1970s,Str_1960s,house_acousticness_mean,hip hop_acousticness_std,pop_liveness_std,dance_liveness_std,r&b_acousticness_std,rap_energy_std,rap_key_std,acoustic_acousticness_std,acoustic_acousticness_mean,acoustic_energy_std,acoustic_key_std,soul_acousticness_std
0,0,0.641282,3.058644,0.467911,4.148403,0.275940,4.428289,0.119650,3.608691,0.275940,4.428289,0.199440,6.131128,-18.000646,0.111308,0.630769,2.056123,0.383051,2.479147,101.045969,0.019284,3.338462,0.643502,0.319263,4.061163,24.0,366.624695,2.736285e-06,42.833333,0.051084,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,3.058644,6.131128,6.131128,3.058644,4.428289,4.428289,3.058644,0.641282,4.428289,4.428289,0.000000
1,1,0.278816,3.805912,0.634392,7.129091,0.596000,5.991548,0.192559,2.928603,0.596000,5.991548,0.164490,7.768494,-9.525804,0.280768,0.509804,1.980676,0.082210,7.627447,122.768255,0.035441,4.000000,5.000000,0.656235,4.076658,330.0,321.435189,3.011912e-06,48.903226,0.066535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [5]:
## train, test split##

# A train/test split is constructed where 90% of the subsample is 
# the train data set and 10% the test data set.

# Set train and test sizes
train_size = 0.9
test_size = 1-train_size

# Function to return random train and test sets
def data_splitter(df, train, validate=False, seed=9001):
    
    if validate:
        np.random.seed(seed)
        perm = np.random.permutation(df.index)
        m = len(df)
        train_end = int(train * m)
        validate_end = int(validate * m) + train_end
        train = df.ix[perm[:train_end]]
        validate = df.ix[perm[train_end:validate_end]]
        test = df.ix[perm[validate_end:]]
        return train, validate, test
    else:
        np.random.seed(seed)
        perm = np.random.permutation(df.index)
        m = len(df)
        train_end = int(train * m)
        train = df.ix[perm[:train_end]]
        test = df.ix[perm[train_end:]]
        return train, test

In [6]:
# Create train and test dataframes from subsample
train_df, test_df = data_splitter(data, train_size)

# Return shapes of train and test dataframes
print("Train Size: {}".format(train_df.shape))
print("Test Size: {}".format(test_df.shape))

Train Size: (1278, 949)
Test Size: (142, 949)


In [8]:
# Median imputation of missing values
imp = Imputer(missing_values='NaN', strategy='median', axis=1)
train_df = pd.DataFrame(imp.fit_transform(train_df), columns=data.columns)
test_df = pd.DataFrame(imp.transform(test_df), columns=data.columns)

In [9]:
train_df = train_df[train_df['Followers'] != 0]
test_df = test_df[test_df['Followers'] != 0]

In [10]:
# Final step: create y_train/x_train and y_test/x_test dataframes

# Initialize the training data
y_train = np.log(train_df['Followers'])
x_train = train_df.drop('Followers', axis=1)

# Initialize the testing data
y_test = np.log(test_df['Followers'])
x_test = test_df.drop('Followers', axis=1)

## Baseline models 

In this section, our goal is to examine the baseline performance of simple models on the test set. We would use these baseline test set r2 score as a reference for building more complex models. The models included in this section are mostly multilinear regression models with different subset of predictors and possible polynomial/interaction terms. PCA and LASSO/RRIDGE are explored here as well.

### Part(a) Multilinear regression model with all predictors

The test r2 score for multilinear regression model with all predictors is -2.779. This is evidence suggesting that we might be overfitting our model with too many predictors. Therefore, going forward (part (e)), we would like to fit a regression model with only significant predictors in this model.

In [91]:
X=sm.add_constant(x_train)
X_test=sm.add_constant(x_test)
model=sm.OLS(y_train,X)
results=model.fit()
r2_test_a=r2_score(y_test,results.predict(X_test))
print("For multilinear regression with all terms,R2 score for training set: {}".format(r2_score(y_train,results.predict(X))))
print("For multilinear regression with all terms,R2 score for test set: {}".format(r2_test_a))
#results.summary()

For multilinear regression with all terms,R2 score for training set: 0.7655523815712503
For multilinear regression with all terms,R2 score for test set: -2.779712758436489


###  Part(b) Multilinear regression model with top artists predictors

In our prelimnary EDA anlaysis, we believed that top artists would be a good predictor for the success of a playlist. Therefore, here we fit two models in part (b) and (c) that only include predictors including artists. In part (b), we use the presence of top 30 artists as predictors. As a note, for part (b), top artist are those who appear most often in playlists with 350,000+ followers. With more than 350,000 followers, a playlist will be in the top 20% in terms of followers. 

Our regression generates a test r2 score of 0.017, which is a lot better than -2.8 in part (a). Therefore, there is good reason to consider these predictors in future model building.

In [95]:
top_30_artist_col=['Lil Wayne', 'Van Morrison', 'Galantis',
       'Wiz Khalifa', 'Rihanna', 'Post Malone', 'Axwell /\ Ingrosso',
       'Young Thug', 'JAY Z', 'A$AP Rocky', 'Yo Gotti', 'Chance The Rapper',
       'Led Zeppelin', 'Otis Redding', '21 Savage', 'Deorro', 'Elton John',
       'SZA', 'Ty Dolla $ign', 'Ryan Adams', 'Birdy', 'Miguel', 'Niall Horan',
       'Ellie Goulding', 'Commodores', 'Radiohead', 'SYML', 'First Aid Kit',
       'Lord Huron']

x_train_art=x_train[top_30_artist_col]
x_test_art=x_test[top_30_artist_col]

X1=sm.add_constant(x_train_art)
X2=sm.add_constant(x_test_art)
model2=sm.OLS(y_train,X1)
results2=model2.fit()
print("With top 30 artists,R2 score for test set: {}".format(r2_score(y_test,results2.predict(X2))))
#results2.summary()

With tio 30 artists,R2 score for test set: 0.017373740567596885


In [76]:
#significant artistis
results2.pvalues[results2.pvalues < 0.05].index

Index(['const', 'Galantis', 'Post Malone', 'Yo Gotti', 'Ellie Goulding'], dtype='object')

###  part(c) Multilinear regression model with top artists counts predictors

Top artists are defined different for part (b) and part (c).Here, top artists are ranked by the total number of followers that playlists including this artist aggregate to. For part (c), the predictors are the number of top 10/10-20/20-30/30-40/40-50 artists that a playlist has.

We see that r2 test result is -0.03.

In [66]:
x_train_art_count=x_train[top_artist_count_columns]
x_test_art_count=x_test[top_artist_count_columns]

X3=sm.add_constant(x_train_art_count)
X4=sm.add_constant(x_test_art_count)
model3=sm.OLS(y_train,X3)
results3=model3.fit()
print("With top artists count predictors,R2 score for test set: {}".format(r2_score(y_test,results3.predict(X4))))
results3.summary()

With only the significant terms from the last model,R2 score for test set: -0.031235636069632644


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Followers   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     3.204
Date:                Tue, 05 Dec 2017   Prob (F-statistic):            0.00702
Time:                        00:19:12   Log-Likelihood:                -3166.5
No. Observations:                1257   AIC:                             6345.
Df Residuals:                    1251   BIC:                             6376.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.6438      0.097     99.900      0.000       9.454       9.833
top_0_10       0.1075      0.274      0.392      0.695      -0.431       0.646
top_10_20      0.2819      0.358      0.788      0.431      -0.420       0.984
top_20_30     -0.1965      0.280     -0.702      0.483      -0.745       0.352
top_30_40      0.7624      0.273      2.789      0.005       0.226       1.299
top_40_50      0.7486      0.321      2.330      0.020       0.118       1.379
==============================================================================
Omnibus:                       79.718   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.686
Skew:                          -0.595   Prob(JB):                     6.73e-19
Kurtosis:                       2.574   Cond. No.                         4.32
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

###  Part(d) Multilinear regression model with genre predictors

In our prelimnary EDA anlaysis, we also believed that genres would be a good predictor for playlist followers. Therefore, we fit a regression model with only genre predictors. Here，each predictors is a categorical variable indicating whether the playlist belongs to a specified genre.

We see that test r2 score is -3.785. This could be the result of overfitting since we have 865 genre columns. A subset of genre predictors could still be important and will be considered for building future models.

In [97]:
x_train_genre=x_train[genre_columns]
x_test_genre=x_test[genre_columns]

X5=sm.add_constant(x_train_genre)
X6=sm.add_constant(x_test_genre)
model4=sm.OLS(y_train,X5)
results4=model4.fit()
print("With only the significant terms from the last model,R2 score for test set: {}".format(r2_score(y_test,results4.predict(X6))))
#results4.summary()
len(genre_columns)

With only the significant terms from the last model,R2 score for test set: -3.7847086814421367


865

### Part (e) Multilinear regression with only siginifcant predictors from part (a)

In part(e), we fit a model with siginificant predictors from model in part(a) to reduce overfitting. We have a total of 49 predictors (cut down from 949 originally).

We see that test r2 score goes up to 0.085, which is the best r2 score so far. This indicates that our previous model indeed suffer from overfitting. We should workin on choosing a subset of original predictors as predictors for furture models.

In [98]:
sig_preds=results.pvalues[results.pvalues < 0.05].index
len(sig_preds)
sig_preds

Index(['instrumentalness_mean', 'liveness_mean', 'loudness_std', 'speech_mean',
       'time_std', 'valence_mean', ' 'bass music'', ' 'big band'',
       ' 'christian punk'', ' 'country gospel'', ' 'crunk'', ' 'deep house'',
       ' 'dubstep'', ' 'ectofolk'', ' 'electro house'', ' 'escape room'',
       ' 'experimental'', ' 'filter house'', ' 'garage rock'', ' 'latin pop'',
       ' 'modern blues'', ' 'modern country rock'', ' 'new orleans jazz'',
       ' 'pop emo'', ' 'pop punk'', ' 'progressive electro house'',
       ' 'progressive house'', ' 'progressive uplifting trance'',
       ' 'traditional folk'', ' 'trip hop'', ''alternative rock'',
       ''austindie'', ''blues-rock'', ''canadian metal'', ''chillhop'',
       ''columbus ohio indie'', ''dance-punk'', ''deep new americana'',
       ''edm'', ''g funk'', ''indie punk'', ''pop'',
       ''progressive post-hardcore'', ''speed garage'', 'Radiohead',
       'Str_Best', 'Str_Acoustic', 'Str_2000s', 'dance_liveness_std'],
      dty

In [101]:
#fit a multilinear regression model with significant predictors
x_train2 = x_train[sig_preds]
x_test2 = x_test[sig_preds]

X7=sm.add_constant(x_train2)
X8=sm.add_constant(x_test2)
model5=sm.OLS(y_train,X7)
results5=model5.fit()
r2_test_e=r2_score(y_test,results5.predict(X8))
print("With only the significant terms from the last model,R2 score for test set: {}".format(r2_test_e))
results5.summary()

With only the significant terms from the last model,R2 score for test set: 0.0825894720034378


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Followers   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     7.066
Date:                Tue, 05 Dec 2017   Prob (F-statistic):           1.25e-39
Time:                        10:54:47   Log-Likelihood:                -3016.0
No. Observations:                1257   AIC:                             6132.
Df Residuals:                    1207   BIC:                             6389.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                              15.2241      0.950     16.027      0.000      13.361      17.088
instrumentalness_mean              -3.1447      1.089     -2.887      0.004      -5.282      -1.008
liveness_mean                     -12.1780      2.358     -5.164      0.000     -16.804      -7.552
loudness_std                        3.3418      1.299      2.572      0.010       0.793       5.891
speech_mean                        -2.5326      0.971     -2.608      0.009      -4.438      -0.627
time_std                         -1.25e-08   3.99e-09     -3.131      0.002   -2.03e-08   -4.67e-09
valence_mean                       -7.3241      1.413     -5.183      0.000     -10.096      -4.552
 'bass music'                      -0.4514      0.405     -1.116      0.265      -1.245       0.342
 'big band'                         0.2267      0.316      0.718      0.473      -0.393       0.847
 'christian punk'                  -1.1516      0.685     -1.682      0.093      -2.495       0.192
 'country gospel'                  -0.1673      0.326     -0.514      0.607      -0.806       0.471
 'crunk'                            1.4932      0.785      1.901      0.058      -0.048       3.034
 'deep house'                       0.3671      0.299      1.226      0.220      -0.220       0.954
 'dubstep'                          0.6460      0.540      1.196      0.232      -0.414       1.706
 'ectofolk'                        -2.1576      1.148     -1.880      0.060      -4.409       0.094
 'electro house'                   -0.2006      0.233     -0.859      0.390      -0.659       0.257
 'escape room'                      0.6030      0.182      3.308      0.001       0.245       0.961
 'experimental'                     0.1237      0.213      0.580      0.562      -0.295       0.543
 'filter house'                     1.0122      0.338      2.993      0.003       0.349       1.676
 'garage rock'                      0.2319      0.201      1.154      0.249      -0.163       0.626
 'latin pop'                        0.4265      0.220      1.942      0.052      -0.004       0.857
 'modern blues'                     0.5259      0.187      2.809      0.005       0.159       0.893
 'modern country rock'              0.4418      0.190      2.321      0.020       0.068       0.815
 'new orleans jazz'                -0.4096      0.363     -1.128      0.260      -1.122       0.303
 'pop emo'                         -0.1501      0.209     -0.719      0.472      -0.560       0.259
 'pop punk'                         0.4633      0.185      2.504      0.012       0.100       0.826
 'progressive electro house'        0.5527      0.332      1.666      0.096      -0.098       1.203
 'progressive house'               -0.3463      0.331     -1.048      0.295      -0

### Part (f) Bootstrapping for 10% Predictors

In part (e), we observe that a smaller subset of original predictors may do a lot better in terms of test set prediction. Therefore, in part (f), we randomly choose 10% of predictors and fit a regression model. We do 500 iterations and record corresponding r2 test core and the associated predictors.

We achieve a r2 test score of 0.21. However,since we are just randomly choosing predictors, this result could come from chance alone and may not be very robust.

In [77]:
##bootstrapping for10% predictors
r2_test=[]
pred=[]
for i in range(500):
    train_col=[]
    while len(train_col)==0:
        for ele in x_train.columns:
            u=np.random.uniform(0,1)
            if u>0.9:
                if ele!='Followers':
                    train_col.append(ele)
    pred.append(train_col)
    x_train1 = x_train[train_col]
    x_test1 = x_test[train_col]
    multi2 =LinearRegression(fit_intercept=True)# no need to add constant when doing it this way
    multi2.fit(x_train1, y_train)

    r2_test.append(multi2.score(x_test1, y_test))

In [78]:
def findLargest(r2):
    largest=r2[0]
    count=0
    for i in range(len(r2)):
        if r2[i]>largest:
            largest=r2[i]
            count=i
    return count

In [106]:
print("After bootstrapping for 10% of predictors, the best R2 score for test set: {}".format(r2_test[findLargest(r2_test)]))
len(pred[findLargest(r2_test)])
print('The assocaited predictors are')
pred[findLargest(r2_test)]

After bootstrapping for 10% of predictors, the best R2 score for test set: 0.21855216986291626
The assocaited predictors are


['acousticness_mean',
 'liveness_std',
 'loudness_mean',
 'valence_mean',
 'followers_std',
 'top_0_10',
 " 'ambient'",
 " 'bebop'",
 " 'bluegrass'",
 " 'bow pop'",
 " 'chillhop'",
 " 'classic rock'",
 " 'classical piano'",
 " 'contemporary jazz'",
 " 'country dawn'",
 " 'deep talent show'",
 " 'desi hip hop'",
 " 'ethereal wave'",
 " 'freestyle'",
 " 'future garage'",
 " 'garage punk blues'",
 " 'heavy alternative'",
 " 'indie dream pop'",
 " 'indie garage rock'",
 " 'indie psych-rock'",
 " 'intelligent dance music'",
 " 'latin metal'",
 " 'metropopolis'",
 " 'modern uplift'",
 " 'motown'",
 " 'nashville sound'",
 " 'neo-industrial rock'",
 " 'new americana'",
 " 'noise rock'",
 " 'nu gaze'",
 " 'ok indie'",
 " 'piano blues'",
 " 'pop christmas'",
 " 'pop'",
 " 'power metal'",
 " 'progressive house'",
 " 'progressive post-hardcore'",
 " 'retro electro'",
 " 'scorecore'",
 " 'sludge metal'",
 " 'southern soul'",
 " 'swedish indie rock'",
 " 'teen pop'",
 " 'trance'",
 " 'triangle indie

### Part(g) PCA

In order to cut down the number of predictors, we implement PCA here. We choose number of PCA components from 1 to 100 and choose the optimal number of PCA components.

We acheive the best r2 test score of 0.13 with 30 PCA components.

In [107]:
from sklearn.decomposition import PCA
r2_test_pca=[]
for i in range(1,100):
    pca = PCA(n_components=i)
    pca.fit(x_train)
    x_train_pca = pca.transform(x_train)
    x_test_pca = pca.transform(x_test)
    pca_regression_model = LinearRegression(fit_intercept=True)
    pca_regression_model.fit(x_train_pca, y_train)
    r2_test_pca.append(pca_regression_model.score(x_test_pca, y_test))

0.13027619329603601

In [109]:
print("After PCA, the best R2 score for test set: {}".format(r2_test_pca[findLargest(r2_test_pca)]))
print("The optimal number of components is: {}".format(findLargest(r2_test_pca)+1))

After PCA, the best R2 score for test set: 0.130276193296036
The optimal number of components is: 30


### Part(h) Lasso and Ridge

In part (h), we fit Ridge and Lasso with cross validation and with lamda ranging from 1e^-5 to 10^5.

With Lasso, the test r2 score is 0.109.

With Ridge, the test r2 score is 0.112.

In [111]:
#lasso CV
lambda_list=[pow(10,i) for i in range(-5,5)]
lasso_regression = LassoCV(alphas=lambda_list, fit_intercept=True)
lasso_regression.fit(x_train, y_train)
print("With lasso, R2 score for test set: {}".format(lasso_regression.score(x_test,y_test)))

With lasso, R2 score for test set: 0.1094129599174094


In [112]:
#ridge
ridge_regression = RidgeCV(cv=10,alphas=lambda_list, fit_intercept=True, normalize=True)
ridge_regression.fit(x_train, y_train)
print("With ridge, R2 score for test set: {}".format(ridge_regression.score(x_test,y_test)))

With ridge, R2 score for test set: 0.11234514421712671


### Summary

Since we have 949 predictors in our original data, we encounter the problem of overfitting when constructing regression models. Therefore, for basline models, we focus on selecting a subgroup of predictors that perform well in terms of r2 test score. By using only the significant predictors from the full model, test r2 score improves a lot (from -2.7 to 0.8). PCA gives us r2 score of 0.13. Lasso gives up 0.109 and Ridge gives us 0.11. We should note that iteratively choosing 10% of predictors gives us the best baseline r2 metric (0.21). However, this model is not robust and tend toward overfitting since we do not methodologically choose the predictors. Still, we could use 0.21 as a reference when evaluating our future models. In summary, the most robust and predictive baseline model is PCA with r2 socre of 0.13. But we may use 0.21 as baseline r2_test metric for the furture. In addition, we should consider including top 30 artists predictors since they alone gives us 0.03 r2 test metric. On the other hand, we should carefully select which genre predictors to include since there are 865 of them and would lead to overfitting.